# Analyzing Power Outages

**Name(s)**: Ethan Lau

**Website Link**: (your website link)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Introduction

https://nghosh24.github.io/power-outages/
cd "projects\04-The Data Science Lifecycle"

### 1.1: General Overview
- This project centralizes around the power outages dataset which contains a listing of 1,534 major power outages occuring between January of 2000 to July 2016. The outages reported in this data file affected a single U.S. state at the time of each outage occurrence.

### 1.2: Research Question
- The primary question of research to which this project will analyze how geographical and urban charteristics of each state effect the restoration time of a power outage. Investigating how different demographic factors (such as urban vs. rural populations, population density, and percentage of urban population) correlate with power outage impacts (customers affected, duration) can help in understanding which communities are most vulnerable and need targeted interventions.

| Selected Column Name   | Description                                    | Variable Type   |
|-------------------------|------------------------------------------------|-----------------|
| OBS                     | Index                                          | Index           |
| YEAR                    | Indicates the year when the outage event occurred | Numeric         |
| MONTH                   | Indicates the month when the outage event occurred | Numeric         |
| U.S._STATE              | The U.S. state where the outage occurred      | Categorical     |
| POSTAL.CODE             | Postal code of the affected area               | Categorical     |
| NERC.REGION             | NERC (North American Electric Reliability Corporation) region code of the affected area | Categorical |
| CLIMATE.REGION          | U.S. Climate regions as specified by National Centers for Environmental Information | Categorical     |
| ANOMALY.LEVEL           | This represents the oceanic El Niño/La Niña (ONI) index referring to the cold and warm episodes by season | Numeric |
| CLIMATE.CATEGORY        | This represents the climate episodes corresponding to the years | Categorical |
| CAUSE.CATEGORY          | Categories of all the events causing the major power outages | Categorical |
| OUTAGE.DURATION         | Duration of the outage in minutes              | Numeric         |
| CUSTOMERS.AFFECTED      | Number of customers affected by the outage     | Numeric         |
| RES.CUSTOMERS     | Annual number of customers served in the residential electricity sector of the U.S. state | Integer       |
| COM.CUSTOMERS     | Annual number of customers served in the commercial electricity sector of the U.S. state  | Integer       |
| IND.CUSTOMERS     | Annual number of customers served in the industrial electricity sector of the U.S. state  | Integer       |
| TOTAL.CUSTOMERS   | Annual number of total customers served in the U.S. state                                 | Integer       |
| RES.CUST.PCT      | Percent of residential customers served in the U.S. state (in %)                          | Float         |
| COM.CUST.PCT      | Percent of commercial customers served in the U.S. state (in %)                           | Float         |
| IND.CUST.PCT      | Percent of industrial customers served in the U.S. state (in %)                           | Float         |
| POPULATION              | Population in the U.S. state in a year          | Numeric         |
| POPPCT_URBAN            | Percentage of the total population of the U.S. state represented by the urban population | Percentage |
| POPPCT_UC               | Percentage of the total population of the U.S. state represented by the population of the urban clusters | Percentage |
| POPDEN_URBAN            | Population density of the urban areas (persons per square mile) | Numeric         |
| POPDEN_UC               | Population density of the urban clusters (persons per square mile) | Numeric         |
| AREAPCT_URBAN           | Percentage of the land area of the U.S. state represented by the land area of the urban areas | Percentage     |
| AREAPCT_UC              | Percentage of the land area of the U.S. state represented by the land area of the urban clusters | Percentage |
| PCT_LAND                | Percentage of land area in the U.S. state as compared to the overall land area in the continental U.S. | Percentage     |


## Data Cleaning and Exploratory Data Analysis

### 2.1: Data Cleaning
The process for which the data was cleaned is as follows:
1. Raw data xlsx excel file was converted into a CSV file to accomodate pandas' read_csv functionality.
2. Columns regarding outage start and end time/date were combined to form columns 'OUTAGE.START' and 'OUTAGE.RESTORATION' which are pandas timestamp types.
3. Previous columns regarding outage start and end time/date were subsequently dropped
4. All columns not pertinent to research question were dropped. The filtered dataset exists under variable name 'data'
5. A new column was created called AFFECTED_PCT which is the number of affected persons divided by total customers per state

### 2.2: Accessing Missingness
When OUTAGE.DURATION equals zero:
- Given that it would be sensible that out DEMAND.LOSS.MW and CUSTOMERS.AFFECTED would not exist if duration is zero, then any presence of outages being zero in the dataframe are not useful and are removed because an outage lasting 0 duration means it did not occur.
- Given OUTAGE.DURATION is our dependent variable, all NP.NAN values must be removed. Luckily, only 49 are missing.

In regards to the missingness of particular column values from out selected pool of information, eight contained nan values. They were accessed as follows:
- MONTH: Only 9 entries were missing. Given it would be difficult to assign an estimate to when this event would occur, these rows were removed as the number should have a significant effect on the size of the data. This also has the benefit of eliminating missingness of two other columns completely.
- CLIMATE.REGION: Only 5 values are missing. However, they occur only for Hawaii and Alaska given their location outside the relative structure of United States regional categorization. They will be given the terms "Tropical Pacific" and "Artic Pacific" respectively given the significant climate differences of their respect areas relative to the rest of the United States.
- CUSTOMERS.AFFECTED: It's difficult to access the missingness of customers affected. It seems to be not missing at random. To be discussed later.
- POPDEN_UC and POPDEN_RURAL: Both occur simultaneously and are likely missing due to not being collected. Given the minimal impact due to the few number, these were simply removed from the data.



It is important to note that OUTAGE.DURATION has been converted to hours.

In [3]:
# Data Import
fp = Path('data') / 'outage.csv'
raw_data = pd.read_csv(fp).set_index('OBS')

# Data Cleaning
raw_data['OUTAGE.START'] = pd.to_datetime(raw_data['OUTAGE.START.DATE'] + raw_data['OUTAGE.START.TIME'], unit='D', origin='1899-12-30')
raw_data['OUTAGE.RESTORATION'] = pd.to_datetime(raw_data['OUTAGE.RESTORATION.DATE'] + raw_data['OUTAGE.RESTORATION.TIME'], unit='D', origin='1899-12-30')
raw_data['AFFECTED_PCT'] = raw_data['CUSTOMERS.AFFECTED'] / raw_data['TOTAL.CUSTOMERS']

clean_data = raw_data.drop(columns=[
    'OUTAGE.START.DATE',
    'OUTAGE.START.TIME',
    'OUTAGE.RESTORATION.DATE',
    'OUTAGE.RESTORATION.TIME'
    ])

# Column Selection
data = clean_data.loc[:,[
    "YEAR",
    "MONTH",
    "U.S._STATE",
    "POSTAL.CODE",
    "NERC.REGION",
    "CLIMATE.REGION",
    "ANOMALY.LEVEL",
    "CLIMATE.CATEGORY",
    "CAUSE.CATEGORY",
    "OUTAGE.DURATION",
    "CUSTOMERS.AFFECTED",
    "AFFECTED_PCT",
    "RES.CUSTOMERS",
    "COM.CUSTOMERS",
    "IND.CUSTOMERS",
    "TOTAL.CUSTOMERS",
    "RES.CUST.PCT",
    "COM.CUST.PCT",
    "IND.CUST.PCT",
    "POPULATION",
    "POPPCT_URBAN",
    "POPPCT_UC",
    "POPDEN_URBAN",
    "POPDEN_UC",
    "AREAPCT_URBAN",
    "AREAPCT_UC",
    "PCT_LAND"
    ]]

# Accessing Missingness
data = data[data['OUTAGE.DURATION'].notna()] #& (data['OUTAGE.DURATION'] != 0)]
data['OUTAGE.DURATION'] = data['OUTAGE.DURATION'] / 60

data = data[data['MONTH'].notna()]
data[(data['U.S._STATE'] == 'Hawaii') & (data['CLIMATE.REGION'].isna())]['CLIMATE.REGION'] = 'Tropical Pacific'
data = data[data['POPDEN_UC'].notna()]

# Number of Missing
output = data.isna().sum()
#print(output[output > 0])
#print(data.loc[:, output[output > 0].index])

# Outputs
display(data.head())

C:\Users\minds\AppData\Local\Temp\ipykernel_22264\2934512788.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data['U.S._STATE'] == 'Hawaii') & (data['CLIMATE.REGION'].isna())]['CLIMATE.REGION'] = 'Tropical Pacific'


,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,...,COM.CUST.PCT,IND.CUST.PCT,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,51.000000,...,10.644005,0.411181,5348119,73.27,15.28,2279.0,1700.5,2.14,0.6,91.592666
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,0.016667,...,10.791609,0.374820,5457125,73.27,15.28,2279.0,1700.5,2.14,0.6,91.592666
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,50.000000,...,10.687018,0.392361,5310903,73.27,15.28,2279.0,1700.5,2.14,0.6,91.592666
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,42.500000,...,10.682239,0.422355,5380443,73.27,15.28,2279.0,1700.5,2.14,0.6,91.592666
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,29.000000,...,10.811320,0.367005,5489594,73.27,15.28,2279.0,1700.5,2.14,0.6,91.592666


### 1.4: Exploratory Data Analysis
Firstly, our dependent variable is outage duration. This can be calculated by subtracting the outage restoration datetime from the outage start datetime. However, this is redundant given the presence of variable OUTAGE.DURATION—note that both metrics have been verified as being equivalent.

1. Time Considerations:
    - As urbanization increases, the amount of locations requiring power also increase. This auguments the number of possible faliure points. As time increases, there is a noticible increase in the number of outages per year from 2018 onwards.
    - It seems that there are longer outages during the Fall and Winter months of the year on average, likely due to more inclimate weather. There are also higher quantaties of outages during said months of the year.
2. Cause differentiations:
    - When comparing the different causes for outages, weather is the most significant reason in quantity. It is also results on average in longer outages when compared to the other causes therefore this likely needs to be controlled for.
3. Number of Customers Affected:
    - There appears to be a somewhat inverse relationship between number of customers affected and outage duration. Likely, the larer number of customers affected, there is more incentive for the company to want to fix the outage. That being said, it is hardly a strong relationship given the pattern density being quite low
4. Population Density
    - The weight of population affected by an outage seems to have a minimal affect.
    - Similar to # of customers affected, outage duration decreases as percentage of urban clusters affected increases. Similar justification to wanting to more expiendently deal with larger problems results in faster recovery.

In [4]:
# Duration comparison
Y = 'OUTAGE.DURATION'
dur_2 = clean_data['OUTAGE.RESTORATION'] - clean_data['OUTAGE.START']

comparison_df = pd.Series(pd.to_timedelta(clean_data['OUTAGE.DURATION'], unit='m') - dur_2)
#print(comparison_df.round('H').value_counts())

# Graphs
#print(data.columns)

chart1_x = 'YEAR'
fig1 = px.histogram(data, x=chart1_x, title='# of Outages per Year')

chart2_x = 'MONTH'
fig2 = px.box(data, x=chart2_x, y=Y, range_y=[0, 400], title='Outage Duration (outliers above 400 mins not displayed) vs Month of the Year')

chart3_x = 'CAUSE.CATEGORY'
fig3 = px.histogram(data, x=chart3_x, y=Y, title='# of Outages as a result of some category')
fig3_5 = px.box(data, x=chart3_x, y=Y, range_y=[0, 500], title='Outage Duration (outliers above 500 mins not displayed) vs Outage Causes')

chart4_x = 'CUSTOMERS.AFFECTED'
fig4 = px.scatter(data, x=chart4_x, y=Y, range_y=[0, 1000], title='Outage Duration vs Customers Affected')

fig4.show()



In [5]:
df = data.copy()

df['quartiled_urban_cluster_pop_percentage'] = pd.qcut(data['POPPCT_UC'], 8, retbins=True, duplicates='drop')[0]
df['quartiled_affected_pct'] = pd.qcut(data['AFFECTED_PCT'], 8, retbins=True, duplicates='drop')[0]

table1 = df.groupby(['CLIMATE.REGION', 'quartiled_affected_pct'])['OUTAGE.DURATION'].mean()
table2 = df.groupby(['CLIMATE.REGION', 'NERC.REGION'])['OUTAGE.DURATION'].mean()
display(table1.unstack())
display(table2.unstack())

quartiled_affected_pct,"(-0.001, 0.00208]","(0.00208, 0.00871]","(0.00871, 0.0164]","(0.0164, 0.0243]","(0.0243, 0.0354]","(0.0354, 0.0633]","(0.0633, 1.055]"
CLIMATE.REGION,,,,,,,
Central,38.614035,37.659091,30.498246,52.065351,42.850667,72.933333,76.238235
East North Central,92.340000,3.194444,65.378986,48.868254,69.616667,80.918750,77.248611
Northeast,3.630159,74.264912,57.497101,62.439506,58.727222,80.087879,85.485000
Northwest,5.533951,14.783333,97.583333,66.000000,39.413333,64.909722,148.096667
South,15.547727,28.059302,45.348246,74.735556,57.126190,58.411905,156.721667
Southeast,69.140351,11.716667,34.024405,14.965686,41.817647,44.206000,52.389216
Southwest,6.611333,1.883333,3.400000,1.755556,40.416667,27.020833,12.979167
West,26.175309,41.262381,19.521795,22.511111,37.860000,52.616667,119.704167
West North Central,0.000000,NaN,160.000000,NaN,NaN,NaN,3.708333


NERC.REGION,ECAR,RFC,SERC,SPP,MRO,NPCC,WECC,TRE,FRCC,"FRCC, SERC"
CLIMATE.REGION,,,,,,,,,,
Central,111.161667,46.691801,22.765432,155.955556,NaN,NaN,NaN,NaN,NaN,NaN
East North Central,84.725926,108.236145,NaN,NaN,47.550926,NaN,NaN,NaN,NaN,NaN
Northeast,NaN,45.147504,NaN,NaN,NaN,54.369501,NaN,NaN,NaN,NaN
Northwest,NaN,NaN,NaN,NaN,NaN,NaN,21.408333,NaN,NaN,NaN
South,87.937500,5.000000,49.619048,40.557602,227.500000,NaN,6.353333,49.342901,NaN,NaN
Southeast,NaN,19.400000,23.736598,NaN,NaN,NaN,NaN,NaN,71.185271,6.2
Southwest,NaN,0.016667,NaN,1.266667,35.666667,NaN,26.588824,NaN,NaN,NaN
West,NaN,NaN,3.716667,NaN,NaN,NaN,27.253676,NaN,NaN,NaN
West North Central,NaN,0.983333,NaN,2.650000,29.383333,NaN,0.727083,NaN,NaN,NaN


## Assessment of Missingness

### 3.1: NMAR
One potential candidate for NMAR could be our dependent variable OUTAGE.DURATION. This variable indicates the duration of power outages, and its missingness might be directly related to the severity or type of the outage itself. For instance, if an outage was particularly short or insignificant, it might not have been reported or recorded with the same rigor as a more severe or longer-lasting outage. This means that the missingness of OUTAGE.DURATION could be inherently related to the value of OUTAGE.DURATION itself, making it NMAR.

To further investigate and possibly reclassify this missingness to MAR, additional data could be gathered on the reporting practices of different utility companies or regions. Understanding the criteria and circumstances under which certain outage durations are left unreported would help in assessing if there's an underlying pattern that explains the missingness.

### 3.2: Missingness Mechanism
We will analyze the missingness of OUTAGE.DURATION. This was done by running a permutation test against NERC.REGION and POPPCT_UC. If regional management affects reporting, this may give reason to possible missingness being a result of companies not wishing to report their failings. The ladder column was selected due to duration affected accuracy.

#### 3.2.1 NERC Region affect on Outage Duration
- Null Hypothesis: The distribution of Cause Category is the same when Duration is missing vs not missing.
- Alternate Hypothesis: The distribution of NERC Region is different when Duration is missing vs not missing.

There is an observed TVD of 0.315 with a pvalue of 0, indicating a strong rejection of the null hypothesis at a 95% confidence level. Therefore, the distribution of NERC Region is different when Duration is missing vs not missing.

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ks_2samp

def permutation_fn(df, target_col, column, N=1000):
    missing_ccn_ages = df[df[target_col].isna()][column]
    present_ccn_ages = df[df[target_col].notna()][column]

    observed_diff = np.array(np.mean(missing_ccn_ages) - np.mean(present_ccn_ages))

    # Permutations
    results = []
    data = df.copy()
    
    for _ in range(N):
        data['temp_perm'] = np.random.permutation(data[target_col])

        temp_missing_ccn_ages = data[data['temp_perm'].isna()][column]
        temp_present_ccn_ages = data[data['temp_perm'].notna()][column]
        
        temp_observed_diff = np.array(np.mean(temp_missing_ccn_ages) - np.mean(temp_present_ccn_ages))
        results.append(temp_observed_diff)

    p_val = np.mean(results >= observed_diff)
    if p_val < 0.05:
        return results, observed_diff, p_val, 'R'
    else:
        return results, observed_diff, p_val, 'NR'
    
def TVD_fn(df, target_col, compare_col):
    missing_duration = df[df[target_col].isna()]
    non_missing_duration = df[df[target_col].notna()]
    
    # Calculate the distributions based on the state column for both missing and non-missing durations
    dist_missing = missing_duration[compare_col].value_counts(normalize=True)
    dist_non_missing = non_missing_duration[compare_col].value_counts(normalize=True)
    
    # Reindex to ensure both distributions have the same index
    all_categories = set(dist_missing.index) | set(dist_non_missing.index)
    dist_missing = dist_missing.reindex(all_categories, fill_value=0)
    dist_non_missing = dist_non_missing.reindex(all_categories, fill_value=0)
    
    tvds =  np.abs(dist_missing - dist_non_missing).sum() / 2
    
    #p_value = np.mean(tvds >= observed_tvd)
    p_value = np.mean(tvds)
    
    if p_value <= 0.05:
        return [p_value, 'R']
    else:
        return [p_value, 'NR']
    
def permutation_test_tvd(dataframe, duration_col, state_col, n_permutations=1000):
    observed_tvd = TVD_fn(dataframe, duration_col, state_col)[0]
    
    permuted_tvds = np.zeros(n_permutations)
    
    for i in range(n_permutations):
        permuted_data = dataframe.copy()
        permuted_data[duration_col] = np.random.permutation(permuted_data[duration_col])
        permuted_tvds[i] = TVD_fn(permuted_data, duration_col, state_col)[0]
    
    p_value = np.mean(permuted_tvds >= observed_tvd)

    if p_value <= 0.05:
        output = 'R'
    else:
        output = 'NR'
    
    return permuted_tvds, observed_tvd, p_value, output

In [7]:
permuted_tvds, observed_tvd, p_value, output = permutation_test_tvd(raw_data, 'OUTAGE.DURATION', 'NERC.REGION', 10000)

df_plot = pd.DataFrame({'TVD': permuted_tvds})
fig_TVD = px.histogram(df_plot, x='TVD', histnorm='probability density', 
                       title='Permutation Test on TVD', opacity=0.5)

fig_TVD.add_vline(x=observed_tvd, line_dash="dash", line_color="red", 
                  annotation_text="Observed TVD", annotation_position="top right")

fig_TVD.show()
print(observed_tvd)
print(p_value)
print(output)

0.3153910849453322
0.0002
R


#### 3.2.2 Customers Affected on Outage Duration
- Null Hypothesis: The distribution of Customers Affected is the same when Duration is missing vs not missing.
- Alternate Hypothesis: The distribution of Customers Affected is different when Duration is missing vs not missing.

There is an observed Difference in Means of -20599.733 with a pvalue of 0.6181, indicating we cannot reject the null hypothesis at a 95% confidence level. Therefore, the distribution of Customers Affected is different when Duration is missing vs not missing.

In [8]:
permuted_tvds, observed_tvd, p_value, output = permutation_fn(raw_data, 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 10000)

df_plot = pd.DataFrame({'Difference in Means': permuted_tvds})
fig_TVD = px.histogram(df_plot, x='Difference in Means', histnorm='probability density', 
                       title='Permutation Test on Difference in Means', opacity=0.5)

fig_TVD.add_vline(x=observed_tvd, line_dash="dash", line_color="red", 
                  annotation_text="Observed Difference in Means", annotation_position="top right")

fig_TVD.show()
print(observed_tvd)
print(p_value)
print(output)

-20599.732900432893
0.625
NR


## Hypothesis Testing

We will test whether the average outage duration differ between different climate regions. The relevant columns for this test are OUTAGE.DURATION and CLIMATE.REGION.
- Null Hypothesis (H0): The average outage duration is the same for all climate regions.
- Alternative Hypothesis (H1): The average outage duration is different for at least one climate region compared to others.

We will use the ANOVA (Analysis of Variance) test to compare the means of outage durations across different climate regions. ANOVA is suitable here because we are comparing the means of more than two groups.

Significance Level
We will use a significance level of 0.05.

The p-value is 0.0002, indicating we have a strong rejection of the null hypothesis at a 95% confidence level. Therefore, the average outage duration is different for at least one climate region compared to others.

In [9]:
from scipy.stats import f_oneway

def anova_permutation_test(df, target_col, column, N=1000):
    grouped_data = df.groupby(column)[target_col].apply(list)
    observed_stat = f_oneway(*grouped_data)[0]

    # Permutations
    results = []
    data = df.copy()
    
    for _ in range(N):
        data['temp_perm'] = np.random.permutation(data[column])

        temp_grouped_data = data.groupby('temp_perm')[target_col].apply(list)
        temp_observed_stat = f_oneway(*temp_grouped_data)[0]

        results.append(temp_observed_stat)

    p_val = np.mean(results >= observed_stat)
    if p_val < 0.05:
        return results, observed_stat, p_val, 'R'
    else:
        return results, observed_stat, p_val, 'NR'


permuted_anova, observed_anova, p_value, output = anova_permutation_test(data, 'OUTAGE.DURATION', 'CLIMATE.REGION', 10000)

df_plot = pd.DataFrame({'ANOVA': permuted_anova})
fig_TVD = px.histogram(df_plot, x='ANOVA', histnorm='probability density', 
                       title='ANOVA Permutation Test on Difference in Means', opacity=0.5)

fig_TVD.add_vline(x=observed_anova, line_dash="dash", line_color="red", 
                  annotation_text="Observed ANOVA", annotation_position="top right")

fig_TVD.show()
print(observed_anova)
print(p_value)
print(output)

6.05473945225524
0.0005
R


## Framing a Prediction Problem

Prediction Problem: Predicting the duration of a power outage (OUTAGE.DURATION) based on various demographic and geographical characteristics of the affected area.

Type: Regression
Response Variable: OUTAGE.DURATION (Duration of the outage in minutes)


Justification for Response Variable:
OUTAGE.DURATION is chosen as the response variable because it directly measures the impact of a power outage in terms of its duration. Understanding the factors influencing outage duration can help utilities and policymakers in allocating resources more effectively for outage management and restoration efforts.


Features for Prediction:

Geographical Characteristics:
- POPULATION (Population in the U.S. state)
- POPPCT_URBAN (Percentage of the total population represented by the urban population)
- POPDEN_URBAN (Population density of the urban areas)
- AREAPCT_URBAN (Percentage of land area represented by the urban areas)
- PCT_LAND (Percentage of land area in the U.S. state)

Demographic Characteristics:
- RES.CUSTOMERS (Annual number of customers served in the residential electricity sector)
- COM.CUSTOMERS (Annual number of customers served in the commercial electricity sector)
- IND.CUSTOMERS (Annual number of customers served in the industrial electricity sector)
- TOTAL.CUSTOMERS (Annual number of total customers served)
- RES.CUST.PCT (Percentage of residential customers served)
- COM.CUST.PCT (Percentage of commercial customers served)
- IND.CUST.PCT (Percentage of industrial customers served)
- POPPCT_UC (Percentage of the total population represented by the population of the urban clusters)
- POPDEN_UC (Population density of the urban clusters)
- AREAPCT_UC (Percentage of land area represented by the urban clusters)

Justification for Features:
These features are chosen because they represent different aspects of geographical and demographic characteristics that may influence the restoration time of a power outage. For instance, population density and the proportion of urban population might affect the ease of access for repair crews and the complexity of the electrical infrastructure in an area, thus impacting outage duration.


Metric for Evaluation: Mean Absolute Error (MAE)
Justification for Metric: MAE is chosen as the evaluation metric because it provides a straightforward interpretation of the average magnitude of errors in the predicted outage duration. It is easy to understand and useful for comparing the performance of different models. Additionally, MAE is less sensitive to outliers compared to other metrics like Mean Squared Error (MSE).

## Baseline Model

For the baseline model, I'll use a simple linear regression model with two features: "POPULATION" and "POPPCT_URBAN". These features represent demographic and geographical characteristics that might influence the duration of a power outage.

Here's how I'll implement the baseline model using scikit-learn:

Feature Transformation: Since "U.S._STATE" and "NERC.REGION" are categorical variables, I'll encode them using one-hot encoding. For simplicity, I'll ignore other categorical variables for this baseline model.

Model Training: I'll use a simple linear regression model to predict the duration of a power outage based on the transformed features.

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

features = ["POPULATION", "POPPCT_URBAN", "COM.CUSTOMERS", "IND.CUSTOMERS", "CLIMATE.REGION"]
target = "OUTAGE.DURATION"

X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

numeric_features = ["POPULATION", "POPPCT_URBAN", "COM.CUSTOMERS", "IND.CUSTOMERS"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ["CLIMATE.REGION"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

baseline_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

mae_baseline = mean_absolute_error(y_test, y_pred)
print("Baseline Model MAE:", mae_baseline)


Baseline Model MAE: 48.25645431091414


## Step 7: Final Model

We will improve upon the baseline model by engineering new features, encoding categorical variables, and performing hyperparameter tuning.

Categorical Encoding:
Encode the CLIMATE.REGION categorical feature using an appropriate method, such as One-Hot Encoding.

Pipeline Implementation:
Use a Pipeline from sklearn to ensure all steps (feature transformation, encoding, model training) are applied consistently.
Hyperparameter Tuning:

Use GridSearchCV to search for the best hyperparameters for the chosen model.
Tune parameters such as tree depth for decision trees or number of estimators for ensemble methods.

Model Evaluation:
Evaluate the final model using the same unseen and seen datasets as in the baseline model to ensure fair comparison.

Model Selection:
- Lasso with and without polynomial features
- Ridge with and without polynomial features

In [29]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge

data['POP_DENSITY_RATIO'] = data['POPDEN_URBAN'] / (data['POPDEN_UC'] + 1)
data['URBAN_CUSTOMERS_RATIO'] = data['COM.CUSTOMERS'] / (data['POPULATION'] + 1)

features = ["POPULATION", "POPPCT_URBAN", "COM.CUSTOMERS", "IND.CUSTOMERS", "CLIMATE.REGION", 
            "POPDEN_URBAN", "POPDEN_UC", "AREAPCT_URBAN", "AREAPCT_UC", "POP_DENSITY_RATIO", "URBAN_CUSTOMERS_RATIO"]
target = "OUTAGE.DURATION"

X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
n_iterations = [1000, 5000, 10000]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ["POPULATION", "POPPCT_URBAN", "COM.CUSTOMERS", "IND.CUSTOMERS", 
                                   "POPDEN_URBAN", "POPDEN_UC", "AREAPCT_URBAN", "AREAPCT_UC", 
                                   "POP_DENSITY_RATIO", "URBAN_CUSTOMERS_RATIO"]),
        ('cat', OneHotEncoder(), ['CLIMATE.REGION'])
    ])

def create_and_tune_model(model, param_grid, poly=False):
    if poly:
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('poly', PolynomialFeatures()),
            ('model', model)
        ])
    else:
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])

    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)
    print(f"Best parameters for {model}: {grid_search.best_params_}")
    print(f"Best negative MAE: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Test MAE: {mae}")
    
    return grid_search.best_estimator_

# Define parameter grids for each model
lasso_param_grid = {
    'model__alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'model__max_iter': n_iterations
}

ridge_param_grid = {
    'model__alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'model__max_iter': n_iterations
}

lasso_poly_param_grid = {
    'poly__degree': [2, 3, 4],
    'model__alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'model__max_iter': n_iterations
}

ridge_poly_param_grid = {
    'poly__degree': [2, 3, 4],
    'model__alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'model__max_iter': n_iterations
}

lasso_model = create_and_tune_model(Lasso(), lasso_param_grid)
ridge_model = create_and_tune_model(Ridge(), ridge_param_grid)

lasso_poly_model = create_and_tune_model(Lasso(), lasso_poly_param_grid, poly=True)
ridge_poly_model = create_and_tune_model(Ridge(), ridge_poly_param_grid, poly=True)

models = [lasso_model, ridge_model, lasso_poly_model, ridge_poly_model]
model_names = ["Lasso", "Ridge", "Lasso with Poly", "Ridge with Poly"]

best_model = None
best_mae = float('inf')
best_model_name = ""

for model, name in zip(models, model_names):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    if mae < best_mae:
        best_mae = mae
        best_model = model
        best_model_name = name

print(f"Best model: {best_model_name} with Test MAE: {best_mae}")

c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+05, tolerance: 6.919e+02

c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.573e+05, tolerance: 8.305e+02

c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.499e+05, tolerance: 8.728e+02

c:\Users\minds\Miniforge3\envs\

Best parameters for Lasso(): {'model__alpha': 0.01, 'model__max_iter': 1000}
Best negative MAE: -44.87704540751562
Test MAE: 48.60597930387921
Best parameters for Ridge(): {'model__alpha': 1, 'model__max_iter': 1000}
Best negative MAE: -44.81179979902747
Test MAE: 48.524144018933875


c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.389e+06, tolerance: 6.919e+02

c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.899e+06, tolerance: 8.305e+02

c:\Users\minds\Miniforge3\envs\dsc80\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.942e+06, tolerance: 8.728e+02

c:\Users\minds\Miniforge3\envs\

Best parameters for Lasso(): {'model__alpha': 0.1, 'model__max_iter': 10000, 'poly__degree': 3}
Best negative MAE: -43.3826631560196
Test MAE: 47.798467426744395
Best parameters for Ridge(): {'model__alpha': 10, 'model__max_iter': 1000, 'poly__degree': 3}
Best negative MAE: -43.34616818722702
Test MAE: 47.09195461655448
Best model: Ridge with Poly with Test MAE: 47.09195461655448


Best parameters for Lasso(): {'model__alpha': 0.1, 'model__max_iter': 10000, 'poly__degree': 3}
Best negative MAE: -43.3826631560196
Test MAE: 47.798467426744395
Best parameters for Ridge(): {'model__alpha': 10, 'model__max_iter': 1000, 'poly__degree': 3}
Best negative MAE: -43.34616818722702
Test MAE: 47.09195461655448
Best model: Ridge with Poly with Test MAE: 47.09195461655448

## Fairness Analysis

In [66]:
def fairness_finder(numerical_col, threshold_percentile = 50):
    threshold = np.percentile(data[numerical_col], threshold_percentile)

    x_indices = X_test[numerical_col] > threshold
    y_indices = ~x_indices

    y_pred_urban = best_model.predict(X_test[x_indices])
    y_pred_rural = best_model.predict(X_test[y_indices])

    # Observed RMSE
    rmse_urban = np.sqrt(mean_absolute_error(y_test[x_indices], y_pred_urban))
    rmse_rural = np.sqrt(mean_absolute_error(y_test[y_indices], y_pred_rural))
    observed_diff = rmse_urban - rmse_rural

    # Permutation test
    n_permutations = 1000
    perm_diffs = []
    combined_data = np.concatenate([y_pred_urban, y_pred_rural])

    for _ in range(n_permutations):
        np.random.shuffle(combined_data)
        perm_urban = combined_data[:len(y_pred_urban)]
        perm_rural = combined_data[len(y_pred_urban):]
        perm_rmse_urban = np.sqrt(mean_absolute_error(y_test[x_indices], perm_urban))
        perm_rmse_rural = np.sqrt(mean_absolute_error(y_test[y_indices], perm_rural))
        perm_diffs.append(perm_rmse_urban - perm_rmse_rural)

    p_value = (np.abs(perm_diffs) >= np.abs(observed_diff)).mean()

    if p_value < 0.05:
        return perm_diffs, observed_diff, numerical_col, p_value, 'R'
    else:
        return perm_diffs, observed_diff, numerical_col, p_value, 'NR'


numerical_columns = data.select_dtypes(include=['number']).columns.tolist()

unfair_models = []

for col_name in numerical_columns:
    try:
        distribution, observed_diff, column, p_value, rejection = fairness_finder(col_name)
        
        if rejection == 'R':
            unfair_models.append([column, distribution, observed_diff, p_value])
    except:
        continue

To perform a fairness analysis on your final model predicting outage duration, we extracted all numerical columns and performed permutation test where group x was those above the 50th percentile of values and group y was those below for each numerical column. The appropriate evaluation metric is Root Mean Squared Error (RMSE).

Null Hypothesis: The model's RMSE for the numerical column are roughly the same, any differences observed are due to random chance.
Alternative Hypothesis: The model's RMSE for the upper 50th percentile of the numerical column is significantly different (higher) compared to those less than the 50th percentile, indicating potential unfairness in performance.

Numerical Columns Evaluated:
- YEAR
- MONTH
- ANOMALY.LEVEL
- OUTAGE.DURATION
- CUSTOMERS.AFFECTED
- AFFECTED_PCT
- RES.CUSTOMERS
- COM.CUSTOMERS
- IND.CUSTOMERS
- TOTAL.CUSTOMERS
- RES.CUST.PCT
- COM.CUST.PCT
- IND.CUST.PCT
- POPULATION
- POPPCT_URBAN
- POPPCT_UC
- POPDEN_URBAN
- POPDEN_UC
- AREAPCT_URBAN
- AREAPCT_UC
- PCT_LAND
- TOTAL_URBAN_POPULATION
- RES_TO_TOTAL_CUSTOMERS_RATIO
- POP_DENSITY_RATIO
- URBAN_CUSTOMERS_RATIO

The only column which did not pass the fairness test was PODDEN_UC. The observed statistic was 1.419; the p-value is 0.033 at a 95% confidence level, therefore, we reject the null hypothesis. The model's RMSE for population density of urban clusters above 1528.6 is significantly different (higher) compared to those less than 1528.6, indicating potential unfairness in performance.

In [85]:
df_plot = pd.DataFrame({'Z': unfair_models[0][1]})
fig_TVD = px.histogram(df_plot, x='Z', histnorm='probability density', 
                       title='Fairness Test on ' + unfair_models[0][0], opacity=0.5)

fig_TVD.add_vline(x=unfair_models[0][2], line_dash="dash", line_color="red", 
                  annotation_text="Observed Test Stat", annotation_position="top right")

fig_TVD.show()